<a href="https://colab.research.google.com/github/AmberG08/GOSLING_Amber_MEngCapstone/blob/master/vcc_lstm_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
#drive.mount('/content/mnt')
#nb_path = '/content/notebooks'
#os.symlink('/content/mnt/My Drive/Capstone', nb_path)
##sys.path.insert(0, nb_path)  # or append(nb_path)

!pip install scikeras[tensorflow]
!pip install mapie

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer

from scikeras.wrappers import KerasRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from mapie.regression import MapieRegressor

'''Load in file'''
df_main = pd.read_csv('/content/drive/MyDrive/Capstone/CleanedData.csv')

'''Load in file'''
df_main = pd.read_csv('/content/drive/MyDrive/Capstone/CleanedData.csv')
df_main['Timestamp'] = pd.to_datetime(df_main['Timestamp'], format='%d/%m/%Y %H:%M')


'''Getting VCC datapoints'''
df_celldata = pd.DataFrame(df_main, columns=['Timestamp', 'VCC'])

no_nans = df_celldata[~df_celldata.isnull().any(axis=1)]
print(no_nans)


actuals = df_celldata['VCC']

celldata=df_celldata['VCC'].to_numpy()
celldata = celldata.reshape((len(celldata),1))

boxcox = PowerTransformer(method='box-cox')
df_celldata['VCC'] = boxcox.fit_transform(celldata)

'''Separating data'''
time_sep=pd.to_datetime('2021-05-04 9:35:00')
df_log = df_celldata[df_celldata['Timestamp'] <= time_sep]
df_death = df_celldata[df_celldata['Timestamp'] >= time_sep]

df_log = df_log['VCC']
df_death = df_death['VCC']

nan_indices_log = np.isnan(df_log)
coefficients_log = np.polyfit(df_log.index[~nan_indices_log], df_log[~nan_indices_log], 3)
df_log= np.polyval(coefficients_log, df_log.index)
#print(df_log.shape)

nan_indices_death = np.isnan(df_death)
coefficients_death = np.polyfit(df_death.index[~nan_indices_death], df_death[~nan_indices_death], 1)
df_death = np.polyval(coefficients_death, df_death.index)
#print(df_death.shape)

'''Inverse box cox transform'''

""" Concatenation """
df_log = df_log[:-1] # cut out last datapoint to make them connect
df_main['VCC'] = boxcox.inverse_transform(pd.concat([pd.DataFrame(df_log), pd.DataFrame(df_death)], axis=0, ignore_index=True))

plt.plot(df_main.index, df_main['VCC'], label='Boxcox polyval', color='black')
plt.scatter(actuals.index, actuals, label='Boxcox transformed Data', color='black')
plt.xlabel("Index")
plt.ylabel("Box cox VCC")
plt.legend()
plt.show()

'''Cut out intial data''' ##Cut out as required
cut_date_time = pd.to_datetime('2021-04-30 14:45:00')
df_main = df_main[df_main['Timestamp'] >= cut_date_time]
actuals = df_main['VCC']

# plt.plot(df_main.index, df_main['VCC'], label='Boxcox polyval', color='black')
# #plt.scatter(df_celldata.index, df_celldata['VCC'], label='Boxcox transformed Data', color='black' )
# plt.xlabel("Index")
# plt.ylabel("Box cox VCC")
# plt.legend()
# plt.show()

In [ ]:
""" Set features and target """
filtered_df= df_main.drop('Timestamp', axis=1)
X_df = filtered_df[['Pred (X) 3 PV - O2 Sparge','CEDEX - GLC3B', 'Added Volume', 'Total Volume', 'VCC']]
y_df = filtered_df[['VCC']]

from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_df)
y_scaler = MinMaxScaler(feature_range=(0,1)).fit(y_df)

X_df = X_scaler.fit_transform(X_df)
y_df = y_scaler.fit_transform(y_df)

def df_to_X_y(df, window_size):
  df_as_np = df #df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [r for r in df_as_np[i:i+window_size, [0, 1, 2, 3, 4]]]#, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]] #[a] wraps the row in brackets
    X.append(row)
    label = df_as_np[i+window_size][4]
    y.append(label)
  return np.array(X), np.array(y)

print('Windowing Function in progress')

WINDOW_SIZE = 1
X1, y1 = df_to_X_y(X_df, WINDOW_SIZE)
#print(X1[0])
#print(y1)

print('\nWindow Function done')

# split into train and test sets
train_size = int(len(X_df) * 0.70)
test_size = len(X_df) - train_size

X_train, y_train = X1[0:train_size], y1[0:train_size]
X_test, y_test = X1[train_size:len(X_df)], y1[train_size:len(X_df)]


model1 = Sequential() #This is good
model1.add(InputLayer((WINDOW_SIZE, 5))) #InputLayer((time_steps, features)
model1.add(GRU(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear')) #Important

model1.summary()

'''Compiling the model'''
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])

'''Mapie'''
lstm_model=KerasRegressor(build_fn=model1, epochs=10, batch_size=64) #, callbacks=[cp1])
#lstm_model.fit(X_train, y_train)
#print("\nScikeras model fitted")

mapie_regressor = MapieRegressor(estimator=lstm_model, method='plus')
mapie_regressor = mapie_regressor.fit(X_train, y_train)
print("\n Mapie model fitted")





In [ ]:
'''Getting Predictions'''
train_predictions = mapie_regressor.predict(X_train).flatten() # Flatten gets rid of inner brackets
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train})

y_pred, y_pis = mapie_regressor.predict(X_test, alpha=[0.05]) #95% confidence level

comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Lower Bound': y_pis[:, 0, 0], 'Upper Bound': y_pis[:, 1, 0]})
sorted_comparison_df = comparison_df.sort_index()

In [ ]:
train_predictions_copies = np.reshape(train_predictions, (-1, 1))

#train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train})
#print(train_results['Train Predictions'])

train_predictions = y_scaler.inverse_transform(train_predictions_copies)
LowerBound = y_scaler.inverse_transform(np.reshape(y_pis[:, 0, 0], (-1, 1)))
UpperBound = y_scaler.inverse_transform(np.reshape(y_pis[:, 1, 0], (-1, 1)))
y_pred =  y_scaler.inverse_transform(np.reshape(y_pred, (-1, 1)))
y_test =  y_scaler.inverse_transform(np.reshape(y_test, (-1, 1)))

#comparison_df = pd.DataFrame({'Actual': y_test[:,0], 'Predicted': y_pred, 'Lower Bound': y_pis[:, 0, 0], 'Upper Bound': y_pis[:, 1, 0]})
#comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Lower Bound': LowerBound, 'Upper Bound': UpperBound})
#sorted_comparison_df = comparison_df.sort_index()

train_index = pd.DataFrame(data=train_predictions).index
test_index = pd.DataFrame(data=y_test).index

'''Find the maximum index from the training set'''
max_train_index = max(train_index)

'''Shift the indexes for validation and test sets'''
test_index_shifted = [index + max_train_index + 1 for index in test_index]

In [ ]:
train_index = list(train_results.index)
test_index = sorted_comparison_df.index

# Find the maximum index from the training set
max_train_index = max(train_index)

# Shift the indexes for validation and test sets
test_index_shifted = [index + max_train_index + 1 for index in test_index]

In [ ]:
"""Plotting"""

plt.xlabel("Index")
plt.ylabel("VCC (1e6 cells/mL)")


plt.rcParams.update({'font.size': 14})

plt.fill_between(test_index_shifted, LowerBound[:,0], UpperBound[:,0], color = 'orange', label = 'Prediction bands', alpha = 0.5)

#scatter plot of index vs glucose
plt.scatter(train_index, actuals[0:train_size], label='Actual values', color='darkred', s=1)
plt.scatter(train_index, train_predictions, label='Predictions', color='blue', s=1)

plt.scatter(test_index_shifted, actuals[train_size+1:len(X_df)], color='darkred', s=1)
plt.scatter(test_index_shifted, y_pred[:,0], color = 'blue', s=1)


legend = plt.legend()
legend.legend_handles[0]._sizes = [50]
legend.legend_handles[1]._sizes = [30]
plt.show()

from sklearn.metrics import mean_squared_error,  r2_score, mean_absolute_error

R2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
print('\nR2 Score: ', R2)
print('\nMSE: ', MSE)

In [ ]:
# Concatenating actuals
actuals_concat = actuals[train_size+1:len(X_df)]

# Calculating (Upperbound - Lowerbound)/actuals
cp_metric = ((UpperBound[:,0] - LowerBound[:,0]) / actuals_concat)*100

# Outputting minimum, maximum, and mean of the calculation
cp_metric_min = np.min(cp_metric)
cp_metric_max = np.max(cp_metric)
cp_metric_mean = np.mean(cp_metric)

print("Minimum of the calculation:", round(cp_metric_min,3))
print("Maximum of the calculation:", round(cp_metric_max,3))
print("Mean of the calculation:", round(cp_metric_mean,3))